In [1]:
import rnn_dataloader
import CLSTM

In [2]:
import torch
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import StepLR
import numpy as np
import os

import matplotlib.pyplot as plt
import matplotlib

In [3]:
'''data'''
batch_size=64
data_slice_length=seqLength=2500
trainDataArgesDict={
    'positive_datapath' : "/public/data1/zhangyx/Projects/RnnDemoCode/data/train_positive_data/",
    'negative_datapath' : "/public/data1/zhangyx/Projects/RnnDemoCode/data/train_negative_data",
    'batch_size' : batch_size,
    'data_slice_length' :data_slice_length,
}
valiDataArgesDict={
    'positive_datapath' : "/public/data1/zhangyx/Projects/RnnDemoCode/data/test_positive_data",
    'negative_datapath' : "/public/data1/zhangyx/Projects/RnnDemoCode/data/test_negative_data",
    'batch_size' : batch_size,
    'data_slice_length' :data_slice_length,
}
train_dataloader=rnn_dataloader.getDataLoader(trainDataArgesDict)
vali_dataloader=rnn_dataloader.getDataLoader(valiDataArgesDict)

/public/data1/zhangyx/Projects/RnnDemoCode/data/train_positive_data/  下文件数量有  1 个
/public/data1/zhangyx/Projects/RnnDemoCode/data/train_negative_data  下文件数量有  1 个
positive_datalen: 1384    negative_datalen 2000
/public/data1/zhangyx/Projects/RnnDemoCode/data/test_positive_data  下文件数量有  1 个
/public/data1/zhangyx/Projects/RnnDemoCode/data/test_negative_data  下文件数量有  1 个
positive_datalen: 154    negative_datalen 2000


In [4]:
from torchinfo import summary
'''model'''
# winLength=2500
# hidden_size=16
# seqLength=1
# stride=20
# num_layers=1
# num_of_workers = 0
# use_pin_memmory = False
# drop_last=True
# model = BnLSTM.bnLSTM_32window(input_size=winLength, hidden_size=hidden_size, max_length=seqLength,  num_layers=num_layers,use_bias=True, batch_first=True, dropout=0.5,num_classes = 2)
# summary(model)
model =CLSTM.get_pretrain_model()
summary(model)

Layer (type:depth-idx)                   Param #
CLSTM                                    --
├─Conv1d: 1-1                            96
├─BatchNorm1d: 1-2                       32
├─SiLU: 1-3                              --
├─Conv1d: 1-4                            13,376
├─BatchNorm1d: 1-5                       128
├─Tanh: 1-6                              --
├─LSTM: 1-7                              33,280
├─LSTM: 1-8                              33,280
├─Linear: 1-9                            53,378
Total params: 133,570
Trainable params: 133,570
Non-trainable params: 0

In [5]:

use_gpu =True
device=1
use_device= ("cuda:"+str(device)) if use_gpu and torch.cuda.is_available() else 'cpu'
model.to(use_device)

CLSTM(
  (conv1): Conv1d(1, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (bn1d1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (acti1): SiLU()
  (conv2): Conv1d(16, 64, kernel_size=(13,), stride=(6,), padding=(5,))
  (bn1d2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (acti2): Tanh()
  (LSTM1): LSTM(64, 64)
  (LSTM2): LSTM(64, 64)
  (fc): Linear(in_features=26688, out_features=2, bias=True)
)

In [6]:
max_epoch=100 #train epoch 训练集训练轮数
optim_lr = 0.01
loss_fn = CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=optim_lr, eps=1e-05)
# optimizer = torch.optim.Adam(model.parameters(), lr=optim_lr, eps=1e-05 ,weight_decay=0)
scheduler = StepLR(optimizer, step_size=10, gamma=0.9)

lossArray = []
accArray = []
lossArrayTest= []
accArrayTest = []
best_model_state_dict = None
bastacc=0

In [7]:
for epoch in range(max_epoch):
    print("下面进行下次循环： ",epoch)
    for batch_idx, (data_batch,label_batch) in enumerate(train_dataloader):
        data_batch,label_batch=data_batch.to(use_device), label_batch.to(use_device)
        outputs = model(data_batch)
        loss = loss_fn(outputs, label_batch)
        acc = 100.0 *(label_batch == outputs.max(dim=1).indices).float().mean()
        lossArray.append(loss.item())
        accArray.append(acc.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
    print("Train epoch:", epoch," batchNum:",batch_idx,"  epochAcc:",sum(accArray[-train_dataloader.__len__():])/train_dataloader.__len__()," epochLoss",sum(lossArray[-train_dataloader.__len__():])/train_dataloader.__len__())
    # with torch.no_grad():
    #     epoch_acc=0 #用于寻找最高准确率的好模型
    #     for val_index, (val_data_batch,val_label_batch) in enumerate(vali_dataloader):
    #         val_data_batch,val_label_batch=val_data_batch.to(use_device),val_label_batch.to(use_device)
    #         val_outputs = model(val_data_batch)
    #         lossTest = loss_fn(val_outputs, val_label_batch)
    #         accTest = 100.0 * (val_label_batch == val_outputs.max(dim=1).indices).float().mean()
    #         epoch_acc+=accTest.item()
    #         lossArrayTest.append(lossTest.item()) #记录交叉验证集中总损失与正确率趋势
    #         accArrayTest.append(accTest.item())
    #     epoch_acc=epoch_acc/(val_index+1) # 从0开始
    #     if bastacc<epoch_acc :  #寻找最大准确率的模型
    #         bastacc=epoch_acc
    #         best_model_state_dict=model.state_dict()
    # print("bastacc",bastacc)
    # print("Vali epoch:",epoch," batchNum:",val_index,"  valAcc:",sum(accArrayTest[-vali_dataloader.__len__():])/vali_dataloader.__len__()," valLoss",sum(lossArrayTest[-vali_dataloader.__len__():])/vali_dataloader.__len__())

下面进行下次循环：  0
Train epoch: 0  batchNum: 51   epochAcc: 67.21754807692308  epochLoss 1.1697039598455796
下面进行下次循环：  1
Train epoch: 1  batchNum: 51   epochAcc: 75.57091346153847  epochLoss 0.5280586641568404
下面进行下次循环：  2
Train epoch: 2  batchNum: 51   epochAcc: 77.0733173076923  epochLoss 0.4919041865147077
下面进行下次循环：  3
Train epoch: 3  batchNum: 51   epochAcc: 79.6875  epochLoss 0.46063331170723987
下面进行下次循环：  4
Train epoch: 4  batchNum: 51   epochAcc: 81.31009615384616  epochLoss 0.4391695639261833
下面进行下次循环：  5
Train epoch: 5  batchNum: 51   epochAcc: 82.12139423076923  epochLoss 0.4224254987560786
下面进行下次循环：  6
Train epoch: 6  batchNum: 51   epochAcc: 82.00120192307692  epochLoss 0.4150330573320389
下面进行下次循环：  7
Train epoch: 7  batchNum: 51   epochAcc: 82.87259615384616  epochLoss 0.40454043877812534
下面进行下次循环：  8
Train epoch: 8  batchNum: 51   epochAcc: 83.35336538461539  epochLoss 0.40587748701755816
下面进行下次循环：  9
Train epoch: 9  batchNum: 51   epochAcc: 83.50360576923077  epochLoss 0.40185

KeyboardInterrupt: 

In [ ]:
# print("lossArray",len(lossArray)," accArray",len(accArray)," lossArrayTest",len(lossArrayTest)," accArrayTest",len(accArrayTest))

# from datetime import datetime
# current_datetime = datetime.now()
# formatted_time = current_datetime.strftime("%H.%M")
# model_savedir="/public/data1/zhangyx/Projects/Project_CpG_signal/scripts/raw_scripts/new_model_scripts/model_test"
# # model_savedir=os.path.join(model_savedir, "mito_C3L1_"+formatted_time+"_epo"+str(max_epoch)+"_bs"+str(batch_size)+"_model")
# model_savedir=os.path.join(model_savedir, "mito_LessC1TEncoder1_"+formatted_time+"_epo"+str(20)+"_bs"+str(batch_size)+"_model")
# print("model_savedir",model_savedir)
# #模型保存
# if not os.path.exists(model_savedir):
# # 如果文件夹不存在，则创建
#     os.makedirs(model_savedir)
# torch.save(best_model_state_dict,model_savedir+"/model.pth" )
# #效果可视化
# lossPlt = plt.figure()
# avgloss_array = [np.mean(lossArray[i:i+train_dataloader.__len__()]) for i in range(0, len(lossArray), train_dataloader.__len__())]
# plt.plot(avgloss_array, color='b', linestyle='-', label="TrainLoss")
# avgtestloss_array = [np.mean(lossArrayTest[i:i+vali_dataloader.__len__()]) for i in range(0, len(lossArrayTest), vali_dataloader.__len__())]
# plt.plot(avgtestloss_array, color='y', linestyle='-',label="TestLoss")
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()
# lossPlt.savefig(model_savedir +"/"+'Loss.pdf')
# plt.clf()

# lossPlt = plt.figure()
# avgacc_array = [np.mean(accArray[i:i+train_dataloader.__len__()]) for i in range(0, len(accArray), train_dataloader.__len__())]
# print(avgacc_array)
# avgtestacc_array = [np.mean(accArrayTest[i:i+vali_dataloader.__len__()]) for i in range(0, len(accArrayTest), vali_dataloader.__len__())]
# plt.plot(avgacc_array, color='b', linestyle='-', label="TrainACC")
# plt.plot(avgtestacc_array, color='y', linestyle='-', label="TestACC")
# plt.xlabel('Epoch')
# plt.ylabel('ACC (%)')
# plt.legend()
# plt.show()
# lossPlt.savefig(model_savedir +"/"+'ACC.pdf')
# plt.clf()

In [ ]:
# np.savez(model_savedir +"/"+'lossArray.npz', array1=lossArray)
# np.savez(model_savedir +"/"+'accArray.npz', array1=accArray)
# np.savez(model_savedir +"/"+'lossArrayTest.npz', array1=lossArrayTest)
# np.savez(model_savedir +"/"+'accArrayTest.npz', array1=accArrayTest)